In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import os

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [23]:
# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_feature_maps_cl2):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=1, padding=1) # CL1
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2) # Mean Pooling Layer
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=num_feature_maps_cl2, kernel_size=3, stride=1, padding=1) # CL2
        self.fc1 = nn.Linear(num_feature_maps_cl2 * 32 * 32, 5) # Assuming the input images are 32x32, adjust if different

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # CL1 -> PL1
        x = self.pool(F.relu(self.conv2(x))) # CL2 -> PL2
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc1(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [24]:
# Data preprocessing and loading
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [25]:
batch_size = 32
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '')

train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [26]:
# Hyperparameters

learning_rate = 0.01
num_epochs = 100
num_feature_maps_cl2 = 4 # Example value, this is a hyperparameter

# Initialize model, loss function, and optimizer
model = SimpleCNN(num_feature_maps_cl2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [27]:
# Define the early stopping threshold
threshold = 0.0001  # Define a suitable threshold for your problem
previous_loss = float('inf')  # Initialize with infinity to ensure the first epoch is not stopped

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Calculate the average loss for this epoch
    average_loss = running_loss / len(train_loader)
    
    # Check if the change in loss is below the threshold
    if abs(previous_loss - average_loss) < threshold:
        print(f'Early stopping at epoch {epoch + 1}')
        break
    
    # Update the previous loss
    previous_loss = average_loss

    print(f'[Epoch {epoch + 1}] loss: {average_loss:.4f}')


[Epoch 1] loss: 1.3913
[Epoch 2] loss: 1.0493
[Epoch 3] loss: 0.7186
[Epoch 4] loss: 0.4413
[Epoch 5] loss: 0.3043
[Epoch 6] loss: 0.2641
[Epoch 7] loss: 0.1705
[Epoch 8] loss: 0.1163
[Epoch 9] loss: 0.1326
[Epoch 10] loss: 0.1067
[Epoch 11] loss: 0.0923
[Epoch 12] loss: 0.1059
[Epoch 13] loss: 0.1099
[Epoch 14] loss: 0.1387
[Epoch 15] loss: 0.0751
[Epoch 16] loss: 0.0430
[Epoch 17] loss: 0.0170
[Epoch 18] loss: 0.0067
[Epoch 19] loss: 0.0032
[Epoch 20] loss: 0.0019
[Epoch 21] loss: 0.0015
[Epoch 22] loss: 0.0012
[Epoch 23] loss: 0.0010
[Epoch 24] loss: 0.0009
Early stopping at epoch 25


In [28]:
# Evaluating the model on validation data
model.eval()
correct = 0
total = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Store labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

print(f'Train accuracy: {100 * correct / total:.2f}%')

# # Compute the confusion matrix
# cm = confusion_matrix(all_labels, all_predictions)

# # Plot the confusion matrix as a heatmap
# plt.figure(figsize=(10, 7))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=True, yticklabels=True)
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix for train')
# plt.show()

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in valid_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Validation accuracy: {100 * correct / total:.2f}%')

all_labels = []
all_predictions = []

# Testing the model
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

print(f'Test accuracy: {100 * correct / total:.2f}%')

# # Compute the confusion matrix
# cm = confusion_matrix(all_labels, all_predictions)

# # Plot the confusion matrix as a heatmap
# plt.figure(figsize=(10, 7))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=True, yticklabels=True)
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix for Test')
# plt.show()

Train accuracy: 100.00%
Validation accuracy: 48.60%
Test accuracy: 45.80%
